# Proposed Model

# **Biblioheque**

In [33]:
import random
import numpy as np
from scipy.stats import rice
import pickle
# import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import sys
import timeit
import os

torch.set_default_device('cuda')
torch.set_default_dtype(torch.float64)

# class to save results in file

In [34]:
class Record:
    def __init__(self, TextName):
        self.out_file = open(TextName, 'a')
        self.old_stdout = sys.stdout
        sys.stdout = self

    def write(self, text):
        self.old_stdout.write(text)
        self.out_file.write(text)

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self.old_stdout

# **slicer the data**

In [35]:
def slicer(data):
    dataI = data[slice(0, len(data), 2)]
    dataQ = data[slice(1, len(data), 2)]
    return(dataI, dataQ)

# **Modulation**

In [36]:
def mapper_16QAM(QAM16, data):
    map0 = 2*data[slice(0, len(data), 2)] + data[slice(1, len(data), 2)]
    map0 = list(map(int, map0))
    dataMapped = []
    for i in range(len(map0)):
        dataMapped.append(QAM16[map0[i]])
    return(dataMapped)

In [37]:
def calculate_bits(Modulation,NumSubcarriers,NumDataSymb):
    if Modulation=='QPSK':
        Nbpscs=2
    elif Modulation=='16QAM':
        Nbpscs=4
    return NumDataSymb*NumSubcarriers*Nbpscs


# **generate noise**

In [38]:
def AWGN(IFsig, SNR):
    dP = np.zeros(len(IFsig))
    P = 0

    for i in range(len(IFsig)):
        dP[i] = abs(IFsig[i])**2
        P = P + dP[i]

    P = P/len(IFsig)
    gamma = 10**(SNR/10)
    N0 = P/gamma
    n = ((N0/2)**(0.5))*np.random.standard_normal(len(IFsig))
    IF_n = np.zeros((len(IFsig),1))

    for i in range(len(IFsig)):
        IF_n[i,:] = IFsig[i] + n[i]

    return(IF_n)

# Generate channel model

In [39]:
def Generate_channel(Nr, Nt, type):
    if (type == 'gauss'):
        return (np.random.normal(size=(Nr,Nt))+1j*np.random.normal(size=(Nr,Nt)))/np.sqrt(2)
    if (type == 'rayleigh'):
        return (np.random.rayleigh(scale=(1/np.sqrt(2)), size=(Nr,Nt)) + 1j*np.random.rayleigh(scale=(1/np.sqrt(2)), size=(Nr,Nt)))/np.sqrt(2)
    if (type == 'rician'):
        b = 1/np.sqrt(2)
        return (rice.rvs(b, size=(Nr,Nt)) + 1j*rice.rvs(b, size=(Nr,Nt)))/np.sqrt(2)

# **Generate Dataset**



In [40]:
DataSet_x   = []  # x dataset after modulation
DataSet_y   = []  # y dataset
DataSet_HH  = []  # H dataset
DataSet_b   = []  # binary dataset
SNR_min_dB  = 0
SNR_max_dB  = 20
step_dB     = 5
num_dB      = int((SNR_max_dB - SNR_min_dB) / step_dB) + 1

SNR         = np.linspace(SNR_min_dB, SNR_max_dB, num=num_dB)


Nt = 8             # Tx: 8
Nr = 64            # Rx: 128
N_samp = 4000


def Gen_dataset(mode, snr, imperfect, N_samp):
    DataSet_x   = []  # x dataset after modulation
    DataSet_y   = []  # y dataset
    DataSet_H   = []
    DataSet_HH  = []

    NumSubcarriers = 1
    Modulation = '16QAM'
    QAM16 = [-1, -0.333, 0.333, 1]
    NumDataSymb = 1
    N_type = 'gauss'

    if mode == 'train':
        for snr in SNR:
            for runIdx in range(0, N_samp):      # ! 20000 x Nt: samples
                H = Generate_channel(Nt, Nr, N_type)
                HH = np.concatenate((np.concatenate((H.real, H.imag), axis=1),
                                    np.concatenate((-H.imag, H.real), axis=1)), axis=0)
                x = np.zeros((2*Nt, NumSubcarriers))
                a = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
                DataRaw = np.zeros((Nt, a))
                for t in range(Nt):
                    #"data symbol generate"
                    NumBits = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
                    bit = np.random.randint(1, 3, NumBits)-1
                    DataRaw[t, :] = bit
                    for j in range(4):
                        DataSet_b.append(bit[j])
                    I = np.zeros((1, a))
                    I[0, :] = DataRaw[t, :]
                    (dataI, dataQ) = slicer(I[0])

                    # Mapper
                    mapI = mapper_16QAM(QAM16, dataI)
                    mapQ = mapper_16QAM(QAM16, dataQ)
                    x[t] = mapI[0]
                    x[t+Nt] = mapQ[0]

                # transpose
                x = x.transpose()

                y_wo_noise = np.matmul(x, HH)

                # noise
                noise = AWGN(y_wo_noise.transpose(), snr)

                y = y_wo_noise + noise.transpose()

                DataSet_x.append(x)    # ! I, Q sample distance by Nt.
                DataSet_y.append(y)                 # ! output sample

                # Imperfect channel: 5%
                # coef = (2*np.random.randint(0,2,size=HH.shape) - 1)
                # HH = HH + coef * HH * 0.05
                DataSet_HH.append(HH)
                DataSet_H.append(H)               # ! Generated channel

    else:
        for runIdx in range(0, N_samp):      # ! 20000 x Nt: samples
            H = Generate_channel(Nt, Nr, N_type)
            HH = np.concatenate((np.concatenate((H.real, H.imag), axis=1),
                                np.concatenate((-H.imag, H.real), axis=1)), axis=0)
            x = np.zeros((2*Nt, NumSubcarriers))
            a = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
            DataRaw = np.zeros((Nt, a))
            for t in range(Nt):
                #"data symbol generate"
                NumBits = calculate_bits(Modulation, NumSubcarriers, NumDataSymb)
                bit = np.random.randint(1, 3, NumBits)-1
                DataRaw[t, :] = bit
                for j in range(4):
                    DataSet_b.append(bit[j])
                I = np.zeros((1, a))
                I[0, :] = DataRaw[t, :]
                (dataI, dataQ) = slicer(I[0])

                # Mapper
                mapI = mapper_16QAM(QAM16, dataI)
                mapQ = mapper_16QAM(QAM16, dataQ)
                x[t] = mapI[0]
                x[t+Nt] = mapQ[0]

            # transpose
            x = x.transpose()

            y_wo_noise = np.matmul(x, HH)

            # noise
            noise = AWGN(y_wo_noise.transpose(), snr)

            y = y_wo_noise + noise.transpose()

            DataSet_x.append(x)    # ! I, Q sample distance by Nt.
            DataSet_y.append(y)                 # ! output sample

            # Imperfect channel: 5%
            DataSet_HH.append(HH)
            DataSet_H.append(H)               # ! Generated channel


    # Shuffle dataset
    random.seed(1)
    temp = list(zip(DataSet_x, DataSet_y, DataSet_H, DataSet_HH))
    random.shuffle(temp)
    DataSet_x, DataSet_y, DataSet_H, DataSet_HH = zip(*temp)

    return DataSet_x, DataSet_y, DataSet_H, DataSet_HH

In [41]:
def reconstruct_channel (H):
# H_raw = [R(H) I(H); -I(H) R(H)]
# we have four version of H_est
    H_est_1 = []
    H_est_2 = []
    H_est_3 = []
    H_est_4 = []

    H_est_Re_1 = H[0:Nt, 0:Nr]
    H_est_Im_1 = H[0:Nt, Nr:2*Nr]
    H_est_Im_2 = - H[Nt:2*Nt, 0:Nr]
    H_est_Re_2 = H[Nt:2*Nt, Nr:2*Nr]

    H_est_1 = H_est_Re_1 + 1j * H_est_Im_1
    H_est_2 = H_est_Re_1 + 1j * H_est_Im_2
    H_est_3 = H_est_Re_2 + 1j * H_est_Im_1
    H_est_4 = H_est_Re_2 + 1j * H_est_Im_2

    return H_est_1, H_est_2, H_est_3, H_est_4

In [42]:
# def NMSE(H_est, H_raw):
#     H_est_1, H_est_2, H_est_3, H_est_4 = reconstruct_channel(H_est)
#     H_est_vec_1 = torch.reshape(H_est_1, [Nt * Nr, 1])
#     H_est_vec_2 = torch.reshape(H_est_2, [Nt * Nr, 1])
#     H_est_vec_3 = torch.reshape(H_est_3, [Nt * Nr, 1])
#     H_est_vec_4 = torch.reshape(H_est_4, [Nt * Nr, 1])

#     H_raw_vec = torch.reshape(H_raw, [Nt * Nr, 1])

#     mse_1       = (torch.norm(H_raw_vec - H_est_vec_1)**2) / len(H_raw_vec)
#     mse_2       = (torch.norm(H_raw_vec - H_est_vec_2)**2) / len(H_raw_vec)
#     mse_3       = (torch.norm(H_raw_vec - H_est_vec_3)**2) / len(H_raw_vec)
#     mse_4       = (torch.norm(H_raw_vec - H_est_vec_4)**2) / len(H_raw_vec)

#     sigEner   = torch.norm(H_raw_vec)**2

#     nmse_1      = mse_1 / sigEner
#     nmse_2      = mse_2 / sigEner
#     nmse_3      = mse_3 / sigEner
#     nmse_4      = mse_4 / sigEner

#     # Best nmse
#     nmse        = min([nmse_1, nmse_2, nmse_3, nmse_4])

#     return torch.abs(nmse)

In [43]:
def NMSE(H_est, H_raw):
    H_est_1, H_est_2, H_est_3, H_est_4 = reconstruct_channel(H_est)

    # Lấy phần thực của các tensor nếu chúng là complex
    H_est_vec_1 = torch.reshape(H_est_1, [Nt * Nr, 1]).abs()
    H_est_vec_2 = torch.reshape(H_est_2, [Nt * Nr, 1]).abs()
    H_est_vec_3 = torch.reshape(H_est_3, [Nt * Nr, 1]).abs()
    H_est_vec_4 = torch.reshape(H_est_4, [Nt * Nr, 1]).abs()

    H_raw_vec = torch.reshape(H_raw, [Nt * Nr, 1]).abs()

    mse_1 = (torch.norm(H_raw_vec - H_est_vec_1)**2) / len(H_raw_vec)
    mse_2 = (torch.norm(H_raw_vec - H_est_vec_2)**2) / len(H_raw_vec)
    mse_3 = (torch.norm(H_raw_vec - H_est_vec_3)**2) / len(H_raw_vec)
    mse_4 = (torch.norm(H_raw_vec - H_est_vec_4)**2) / len(H_raw_vec)

    sigEner = torch.norm(H_raw_vec)**2

    nmse_1 = mse_1 / sigEner
    nmse_2 = mse_2 / sigEner
    nmse_3 = mse_3 / sigEner
    nmse_4 = mse_4 / sigEner

    # Chọn NMSE tốt nhất
    nmse = min([nmse_1, nmse_2, nmse_3, nmse_4])

    return torch.abs(nmse)

In [44]:
def Input_ISDNN(mode, DataSet_x, DataSet_y, DataSet_H, DataSet_HH, N_samp):
    H_in = []        # ! H_in    , np.diag(np.diag()) return a diag matrix instead of diag components.
    H_true = []   # ! generated s
    H_raw = []
    v = []        # ! vector errors
    xTx = []
    xTy = []
    # steering = [] # ! Steering vector: ZoA and AoA

    if mode == 'train':
        n_sample = N_samp * len(SNR)
    else:
        n_sample = N_samp

    for i in range (n_sample):
        H_true.append(torch.tensor(DataSet_HH[i]))
        H_raw.append(torch.tensor(DataSet_H[i]))
        xTy.append(torch.tensor(np.dot(DataSet_x[i].transpose(), DataSet_y[i])))
        H_in.append(torch.zeros([2*Nt, 2*Nr]))
        v.append(torch.zeros([2*Nt, 2*Nr]))
        xTx.append(torch.tensor(np.dot(DataSet_x[i].transpose(), DataSet_x[i])))
        # steering.append(torch.tensor(DataSet_Steering[i]))

    H_true = torch.stack(H_true, dim=0)
    H_raw = torch.stack(H_raw, dim=0)
    H_in = torch.stack(H_in, dim=0)
    v = torch.stack(v, dim=0)
    xTx = torch.stack(xTx, dim=0)
    xTy = torch.stack(xTy, dim=0)
    # steering = torch.stack(steering, dim=0)

    return H_true, H_raw, H_in, v, xTx, xTy

# Model

In [45]:
class xv(nn.Module):
    def __init__(self):
        super(xv, self).__init__()
        self.fc1 = torch.nn.Linear(4*Nr, 2*Nr)
        self.fc2 = torch.nn.Linear(2*Nr, 2*Nr)
        self.fc3 = torch.nn.Linear(2*Nr, 2*Nr)

        self.delta_1 = torch.nn.parameter.Parameter(torch.rand(1))
        self.delta_2 = torch.nn.parameter.Parameter(torch.rand(1))

    def forward(self, H, v, xTx, xTy):

        xTxH = torch.matmul(xTx, H)

        q    = H - self.delta_1 * xTy + self.delta_2 * xTxH

        concat = torch.concat([q, v], 1)

        z    = torch.tanh(self.fc1(concat))

        H_oh = self.fc2(z)

        v    = self.fc3(z)

        return H_oh, v

In [46]:
class model_driven(nn.Module):
    def __init__(self):
        super(model_driven, self).__init__()

        self.layer1=xv()
        self.layer2=xv()
        self.layer3=xv()
        self.layer4=xv()
        self.layer5=xv()

    def forward(self, H_in, v, xTx, xTy):
        H_oh, v = self.layer1(H_in, v, xTx, xTy)
        H       = torch.tanh(H_oh)

        H_oh, v = self.layer2(H_in, v, xTx, xTy)
        H       = torch.tanh(H_oh)

        H_oh, v = self.layer3(H_in, v, xTx, xTy)
        H       = torch.tanh(H_oh)

        H_oh, v = self.layer4(H_in, v, xTx, xTy)

        return H

# Define model, optimizer, and loss function

In [47]:
def def_model():
    model = model_driven()
    loss = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    folder_model = './model/'

    if not os.path.isdir(folder_model):
        os.makedirs(folder_model)

    file_model = folder_model + 'H'
    # if os.path.isfile(file_model):
    #     generator = torch.load(file_model)

    record_file = 'H'
    return model, loss, optimizer, record_file, file_model

# Main program

In [52]:
epoch         = 0
expected_epoch = 20000
num_samp      = N_samp * len(SNR)
best_nmse     = 1e9
early_stop    = 0
best_model    = ''
batch_size    = 1
# Kiểm tra nếu file tĩnh tồn tại
if os.path.exists('dataset_DetNet.pkl'):
    # Nếu tồn tại, tải dữ liệu từ file tĩnh
    with open('dataset_DetNet.pkl', 'rb') as f:
        DataSet_x, DataSet_y, DataSet_H, DataSet_HH, H_true, H_raw, H_in, v, xTx, xTy = pickle.load(f)
    print("Dữ liệu đã được tải từ file tĩnh!")
else:
    # Sinh dữ liệu nếu file tĩnh không tồn tại
    DataSet_x, DataSet_y, DataSet_H, DataSet_HH = Gen_dataset('train', 0, 0, N_samp)
    H_true, H_raw, H_in, v, xTx, xTy = Input_ISDNN('train', DataSet_x, DataSet_y, DataSet_H, DataSet_HH, N_samp)

    # Lưu dữ liệu để lần sau không phải sinh lại
    with open('dataset_DetNet.pkl', 'wb') as f:
        pickle.dump((DataSet_x, DataSet_y, DataSet_H, DataSet_HH, H_true, H_raw, H_in, v, xTx, xTy), f)
    print("Dữ liệu đã được sinh và lưu lại!")

print("Begin training...")
starttime = timeit.default_timer()

while(True):
        epoch = epoch + 1

        init_loss = 1e9
        while( epoch == 1 and init_loss > 11000):

                model, loss, optimizer, record_file, file_model = def_model()
                init_loss = 0
                for bs in range (int(num_samp / batch_size)):
                    H_1 = model(
                                 torch.squeeze(H_in[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                                 torch.squeeze(v[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                                 torch.squeeze(xTx[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                                 torch.squeeze(xTy[0 + batch_size * bs:batch_size * (bs+1), :, :]))   # predict output from the model
                    init_loss += loss(H_1, torch.squeeze(H_true[0 + batch_size * bs:batch_size * (bs+1), :, :])).item()
                print(init_loss)

        optimizer.zero_grad()   # zero the parameter gradients
        train_loss = 0
        H_f = torch.empty([num_samp, 2*Nt, 2*Nr])
        for bs in range (int(num_samp / batch_size)):
                H_o = model(
                        torch.squeeze(H_in[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                        torch.squeeze(v[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                        torch.squeeze(xTx[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                        torch.squeeze(xTy[0 + batch_size * bs:batch_size * (bs+1), :, :]))   # predict output from the model
                H_f[0 + batch_size * bs:batch_size * (bs+1), :, :] = H_o
                train_loss = loss(H_o,torch.squeeze(H_true[0 + batch_size * bs:batch_size * (bs+1), :, :]))   # calculate loss for the predicted output
                train_loss.backward()   # backpropagate the loss
                optimizer.step()        # adjust parameters based on the calculated gradients
                for j in range(num_samp):
                  nmse = NMSE(H_f[j], H_raw[j])
                nmse = nmse / num_samp
                torch.save(model.state_dict(), file_model +'_'+ str(epoch) + '.pth')
                best_model = file_model + '.pth'
                with Record(record_file + '_log.txt'):
                    print(epoch, nmse.item(), train_loss.item())

Dữ liệu đã được tải từ file tĩnh!
Begin training...
11177.08907556355
11108.765823015152
10194.776080032507
1 0.0 0.4859828733665573
1 0.0 0.5085558930705015
1 0.0 0.5731607209136205
1 0.0 0.49894288769441175
1 0.0 0.5394429592785857
1 0.0 0.46236646244084756
1 0.0 0.4791313963481459
1 0.0 0.5117303222236769
1 0.0 0.48774499314510883
1 0.0 0.5095364617035968
1 0.0 0.5306340801212651
1 0.0 0.4972949784614992
1 0.0 0.5421815606998563
1 0.0 0.5195179112239279
1 0.0 0.5104854009897789
1 0.0 0.49501958899352
1 0.0 0.5330836691071037
1 0.0 0.5231743254085595
1 0.0 0.43339769429373387
1 0.0 0.47185264958773
1 0.0 0.45797510175583594
1 0.0 0.5025016361726384
1 0.0 0.4803777749897096
1 0.0 0.4799029952132697
1 0.0 0.49894718758954604
1 0.0 0.5135190383369905
1 0.0 0.5039156631277726
1 0.0 0.5165552563520226
1 0.0 0.5408216600888625
1 0.0 0.5309094586532491
1 0.0 0.5271178738592199
1 0.0 0.4935613699348987
1 0.0 0.5531671225767604
1 0.0 0.5119346854545618
1 0.0 0.5200608993655316
1 0.0 0.4958502

KeyboardInterrupt: 

# Test function

# Function to test the model

In [ ]:
#best_model=r'C:\Users\SON\Desktop\ISDNN\Python\Unstructured\model_detnet_10k_4l_1\H_1.pth'

In [ ]:
# from scipy.io import savemat

def test(H_raw, H_in, v, xTx, xTy, N_test, log):
    # Load the model that we saved at the end of the training loop
    model = model_driven()
    model.load_state_dict(torch.load(best_model, map_location=torch.device('cpu')))

    with torch.no_grad():
        H_f = torch.empty([N_test, 2*Nt, 2*Nr])
        for bs in range (int(N_test / 1)):
            H_o = model(
                        torch.squeeze(H_in[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                        torch.squeeze(v[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                        torch.squeeze(xTx[0 + batch_size * bs:batch_size * (bs+1), :, :]),
                        torch.squeeze(xTy[0 + batch_size * bs:batch_size * (bs+1), :, :]))   # predict output from the model
            H_f[0 + batch_size * bs:batch_size * (bs+1), :, :] = H_o

        nmse = 0
        for j in range (N_test):
            nmse += NMSE(H_f[j], H_raw[j])

        nmse = nmse / N_test
        with Record(log):
            print(format(nmse.item(), '.7f'))

In [ ]:
## Generate dataset for test

In [ ]:
def LS(DataSet_x, DataSet_y):
    start = timeit.default_timer()
    for i in range (len(DataSet_x)):
        H_hat = np.matmul(
                    np.matmul(
                        np.linalg.pinv(np.matmul(DataSet_x[i].transpose(), DataSet_x[i])),
                        DataSet_x[i].transpose()),
                        DataSet_y[i])
    print(timeit.default_timer() - start)

In [ ]:
def MMSE(DataSet_x, DataSet_y, noise_dB, H_raw):
    snr_dB = 10 ** (-noise_dB / 10)
    start = timeit.default_timer()
    # H_f = np.empty([len(DataSet_x), 2*Nt, 2*Nr])
    for i in range (len(DataSet_x)):
        H_hat = np.matmul(
                    np.matmul(
                        np.linalg.pinv(np.matmul(DataSet_x[i].transpose(), DataSet_x[i]) + snr_dB * np.eye(2*Nt, 2*Nt)),
                        DataSet_x[i].transpose()),
                        DataSet_y[i])
        # H_f[i, :, :] = H_hat
    print(timeit.default_timer() - start)
    # nmse = 0
    # for j in range (len(DataSet_x)):
    #         # tmp =  H_o[j]
    #         # tmp1 = tmp.numpy()
    #         # savemat('H_est.mat', {'H_o': tmp1})
    #     nmse += NMSE(torch.tensor(H_f[j]), torch.tensor(H_raw[j]))
    # nmse = nmse / len(DataSet_x)
    # print(format(nmse.item(), '.7f'))


In [ ]:
SNR_min_dB  = 0
SNR_max_dB  = 20
step_dB     = 2
num_dB      = int((SNR_max_dB - SNR_min_dB) / step_dB) + 1

SNR         = np.linspace(SNR_min_dB, SNR_max_dB, num=num_dB)
log         = './model/log_test.txt'

N_test = int(num_samp * 30 / 100)

for i in range (100):
    for snr in SNR:
        # with Record(log):
        #     print(snr)
        DataSet_x, DataSet_y, DataSet_H, DataSet_HH = Gen_dataset('test', snr, 0, N_test)
        H_true, H_raw, H_in, v, xTx, xTy = Input_ISDNN('test', DataSet_x, DataSet_y, DataSet_H, DataSet_HH, N_test)

        # LS(DataSet_x, DataSet_y)
        # MMSE(DataSet_x, DataSet_y, snr, H_raw)
        test(H_raw, H_in, v, xTx, xTy, N_test, log)